In [ ]:
using LinearAlgebra, Statistics
using Distributions, Plots, QuantEcon, Interpolations, Parameters
using NLsolve, Random
using Expectations
using DataFrames
using StatsBase
using JLD

In [ ]:
function vfi!(ls; tol = 1e-4, maxit = 3500)
    
    @unpack λ, α, β, τ, w_grid_size, h_grid_size, I_grid_size, s_grid_size, μ_unemployed, μ_employed, μ_laidoff = ls 
    @unpack w_grid, I_grid, h_grid, dist_F,s0_grid, sb_grid, F_probs = ls 
    @unpack V0, Vb, V, V1, Vvb, policy_v0, policy_vb, policy_v = ls
    
    it = 0
    dist = 10.0
    
    V_upd = similar(ls.V)
    V0_upd = similar(ls.V0)
    Vb_upd = similar(ls.Vb)
    Vvb_upd = similar(ls.Vvb)
    V1_upd = similar(ls.V1)
    
    while dist > tol && it < maxit
        it += 1
        print(it) 
        
        copyto!(V_upd, ls.V)
        copyto!(V0_upd, ls.V0)
        copyto!(Vb_upd,ls.Vb)
        copyto!(Vvb_upd,ls.Vvb)
        copyto!(V1_upd,ls.V1)
        
        EV = ls.V
        EV0 = ls.V0
        EVb = ls.Vb
        EVvb = ls.Vvb
        EV1 = ls.V1

        # update value function
        one_step_update!(ls, EV0, EVb, EVvb, EV, EV1)
        
        d1 = maximum(abs(x - y) for (x, y) in zip(V_upd, ls.V))
        d2 = maximum(abs(x - y) for (x, y) in zip(V0_upd, ls.V0))
        d3 = maximum(abs(x - y) for (x, y) in zip(Vb_upd, ls.Vb))
        
        max_dd = max(d1, d2)
        dist = max_dd > d3 ? max_dd : d3
    end
end

In [ ]:
Random.seed!(42)

vfi!(ls)

wv0_bar = zeros(ls.h_grid_size)

for ih in 1:ls.h_grid_size
    for iw in ls.w_grid_size:-1:1
        if ls.policy_v[iw,ih] == 1
            wv0_bar[ih] = ls.w_grid[iw]
        end
    end
end

w_bar = zeros(ls.I_grid_size, ls.h_grid_size)

for iI in 1:ls.I_grid_size
    for ih in 1:ls.h_grid_size
        aimvb = ls.Vb[iI,ih]
        
        for iw in ls.w_grid_size:-1:1
            currentwage = ls.w_grid[iw]
            
            if currentwage * ls.h_grid[ih] < ls.I_grid[iI] * 0.7
                
                if ls.V[iw,ih] > aimvb || ls.V[iw,ih] == aimvb
                    w_bar[iI,ih] = currentwage
                end
                
            else
                
                if ls.policy_v[iw,ih] == 1
                    w_bar[iI,ih] = currentwage
                end
                
            end
        end        
    end
end